# Test Gemini vs Ensemble for MMLU


to do:

-ignore claude, add gemma, palm

In [1]:
import os
import copy
import json
import random
import pandas as pd
from pprint import pprint
from datetime import datetime
from datasets import load_dataset

from utils import Annotate
from config import PALM_CONFIG, GEMINI_CONFIG, CLAUDE_CONFIG

In [2]:
seed = 420
now = datetime.now().strftime("%Y%m%d")

In [3]:
dataset = load_dataset("cais/mmlu", "all")
n_sample = 100
# # take a small sample for dev purposes
dataset = dataset['test'].shuffle(seed=seed).select(range(n_sample))

# # user provided data description
# DESCRIPTION = """
# This is a massive multitask test consisting of multiple-choice questions from various branches of knowledge.
# The test spans subjects in the humanities, social sciences, hard sciences, and other areas that are important for some people to learn.
# To attain high accuracy on this test, models must possess extensive world knowledge and problem solving ability.
# This covers 57 subjects  across STEM, the humanities, the social sciences, and more. 
# It ranges in difficulty from an elementary level to an advanced professional level, and it tests both world knowledge and problem solving ability. 
# Subjects range from traditional areas, such as mathematics and history, to more specialized areas like law and ethics.
# """


In [4]:
gemini_prompt_template = """
<QUESTION>
{datapoint}
</QUESTION>
------------

<CHOICES>
{labels}
</choices>
------------

INSTRUCTION:
- read the above question carefully.
- you are given 4 choices seperated by comma in <CHOICES>.
- take your time and pick the precise correct answer from <CHOICES> for the given <QUESTION>.
- remember that there is always only one correct answer.
- return the exact correct answer from <CHOICES>. Don't provide explanations.
"""

In [5]:
prompt = [gemini_prompt_template.format(datapoint=x['question'],
                                        labels=x['choices']) for x in dataset]
print(len(prompt))

100


In [7]:
models = [
    "gemini",
    "palm",
    # "claude"
    ]

PALM_CONFIG["project_config"]["qpm"] = 150

palm_1 =  copy.deepcopy(PALM_CONFIG)
palm_1['config_name'] = "temp_0.4"

palm_2 =  copy.deepcopy(PALM_CONFIG)
palm_2['config_name'] = "temp_0.9"
palm_2["generation_config"]['temperature'] = 0.9

GEMINI_CONFIG["project_config"]["qpm"] = 100

gemini_1 =  copy.deepcopy(GEMINI_CONFIG)
gemini_1['config_name'] = "-1.0-pro-002"

gemini_2 =  copy.deepcopy(GEMINI_CONFIG)
gemini_2['config_name'] = "-1.5-flash-001"
gemini_2['"model"'] = "gemini-1.5-flash-001"

gemini_3 =  copy.deepcopy(GEMINI_CONFIG)
gemini_3['config_name'] = "-1.0-ultra-001"
gemini_3['"model"'] = "gemini-1.0-ultra-001"



model_config = {
    "gemini": [
        gemini_1,
        gemini_2,
        gemini_3
         ],
    "palm": [
        palm_1, 
        palm_2
        ],
    # "claude": [
    #     CLAUDE_CONFIG
    # ]
}

ann = Annotate()


In [8]:
output_dict = await ann.classification(prompt, models, model_config)

Gathering gemini_-1.0-pro-002 results:   0%|          | 0/100 [00:00<?, ?it/s]2024-05-30 17:31:39,703/Annotate[ERROR]: gemini_-1.0-pro-002 Task 0 failed: Content has no parts.
2024-05-30 17:31:39,704/Annotate[ERROR]: gemini_-1.0-pro-002 Task 99 failed: Content has no parts.
Gathering gemini_-1.5-flash-001 results:   0%|          | 0/100 [00:00<?, ?it/s]2024-05-30 17:31:39,708/Annotate[ERROR]: gemini_-1.5-flash-001 Task 0 failed: Content has no parts.
2024-05-30 17:31:39,709/Annotate[ERROR]: gemini_-1.5-flash-001 Task 99 failed: Content has no parts.
Gathering gemini_-1.0-ultra-001 results:   0%|          | 0/100 [00:00<?, ?it/s]2024-05-30 17:31:39,713/Annotate[ERROR]: gemini_-1.0-ultra-001 Task 0 failed: Content has no parts.
2024-05-30 17:31:39,714/Annotate[ERROR]: gemini_-1.0-ultra-001 Task 99 failed: Content has no parts.
Gathering gemini_-1.0-ultra-001 results: 100%|██████████| 100/100 [00:00<00:00, 46320.31it/s]


In [10]:
llm_response = {}

for k in output_dict.keys():
    llm_response[k] = []
    for idx, r in enumerate(output_dict[k]):
        if r is not None:
            stripped_r = r.strip().strip("'")
            if stripped_r in dataset['choices'][idx]:
                llm_response[k].append(dataset['choices'][idx].index(stripped_r))
            else:
                # Handle case where stripped_r is not found in choices
                llm_response[k].append(None)
        else:
            # Handle None values appropriately
            llm_response[k].append(None)

In [11]:
try:
    os.mkdir(f"./data/{now}")
except:
    pass

with open(f"./data/{now}/llm_response_{n_sample}__{now}.json", "w") as json_file:
    json.dump(llm_response, json_file)

In [23]:
with open(f"./data/{now}/llm_response_{n_sample}__{now}.json", "r") as f:
    llm_response= json.load(f)

In [24]:
llm_response

{'palm_temp_0.4': [2,
  0,
  1,
  3,
  2,
  2,
  1,
  2,
  2,
  0,
  2,
  2,
  None,
  2,
  0,
  1,
  0,
  1,
  2,
  1,
  3,
  2,
  1,
  2,
  0,
  1,
  1,
  3,
  2,
  1,
  3,
  2,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  3,
  2,
  1,
  2,
  None,
  1,
  1,
  1,
  3,
  3,
  2,
  1,
  2,
  3,
  1,
  1,
  0,
  None,
  3,
  2,
  3,
  3,
  1,
  2,
  2,
  3,
  None,
  2,
  3,
  1,
  1,
  None,
  3,
  3,
  1,
  3,
  1,
  3,
  1,
  1,
  2,
  2,
  0,
  1,
  0,
  2,
  2,
  0,
  0,
  1,
  0,
  0,
  2,
  0,
  1,
  0,
  1,
  2,
  2,
  2],
 'palm_temp_0.9': [2,
  0,
  None,
  3,
  None,
  2,
  1,
  2,
  2,
  0,
  2,
  2,
  None,
  2,
  0,
  1,
  None,
  1,
  2,
  1,
  3,
  2,
  1,
  2,
  2,
  1,
  1,
  None,
  2,
  1,
  3,
  2,
  0,
  2,
  1,
  1,
  0,
  2,
  1,
  0,
  3,
  2,
  1,
  2,
  None,
  1,
  3,
  0,
  3,
  3,
  2,
  1,
  0,
  3,
  1,
  1,
  3,
  2,
  3,
  2,
  3,
  3,
  1,
  1,
  2,
  3,
  None,
  1,
  3,
  1,
  1,
  None,
  3,
  3,
  1,
  3,
  0,
  3,
  1,
  1,
  2,
  2,
  0,
  1,
  0,
 

In [29]:
# def replace_none_with_random(data, n):
#     new_data = {}  
#     for key, lst in data.items():
#         new_lst = []  
#         for value in lst:
#             if value is None:
#                 new_lst.append(random.randint(0, n))
#             else:
#                 new_lst.append(value)
#         new_data[key] = new_lst 
#     return new_data

def drop_nones(data):
    return pd.DataFrame.from_dict(llm_response).dropna().astype(int).to_dict(orient="list")


def convert_dict_to_indexed_list(data_dict):
    number_map = {key: index for index, key in enumerate(data_dict.keys())}
    max_len = len(next(iter(data_dict.values())))

    result = []
    for index in range(max_len):
        for key, value_list in data_dict.items():
            value = value_list[index]
            converted_value = value 
            result.append([index, number_map[key], converted_value])
    return result
    

def generate_task_config(response_dict, num_classes):

    num_labels = sum(len(lst) for lst in response_dict.values())
    num_tasks =  len(list(response_dict.values())[0])
    num_labelers = len(response_dict)
    z  = 1/num_classes


    tc = [num_labels, num_labelers, num_tasks, num_classes]
    tc.extend([z] * tc[-1])

    return tc

In [31]:
n_class = 4
llm_response = drop_nones(llm_response)
task_conf = generate_task_config(llm_response, n_class)
llm_result_list = convert_dict_to_indexed_list(llm_response)
llm_result_list.insert(0, task_conf)

In [32]:
llm_result_list

[[395, 5, 79, 4, 0.25, 0.25, 0.25, 0.25],
 [0, 0, 0],
 [0, 1, 0],
 [0, 2, 1],
 [0, 3, 1],
 [0, 4, 1],
 [1, 0, 2],
 [1, 1, 2],
 [1, 2, 3],
 [1, 3, 3],
 [1, 4, 3],
 [2, 0, 1],
 [2, 1, 1],
 [2, 2, 1],
 [2, 3, 1],
 [2, 4, 1],
 [3, 0, 2],
 [3, 1, 2],
 [3, 2, 2],
 [3, 3, 2],
 [3, 4, 2],
 [4, 0, 2],
 [4, 1, 2],
 [4, 2, 0],
 [4, 3, 0],
 [4, 4, 1],
 [5, 0, 0],
 [5, 1, 0],
 [5, 2, 2],
 [5, 3, 2],
 [5, 4, 2],
 [6, 0, 2],
 [6, 1, 2],
 [6, 2, 1],
 [6, 3, 1],
 [6, 4, 1],
 [7, 0, 2],
 [7, 1, 2],
 [7, 2, 2],
 [7, 3, 2],
 [7, 4, 2],
 [8, 0, 2],
 [8, 1, 2],
 [8, 2, 2],
 [8, 3, 2],
 [8, 4, 2],
 [9, 0, 0],
 [9, 1, 0],
 [9, 2, 1],
 [9, 3, 1],
 [9, 4, 1],
 [10, 0, 1],
 [10, 1, 1],
 [10, 2, 1],
 [10, 3, 1],
 [10, 4, 1],
 [11, 0, 1],
 [11, 1, 1],
 [11, 2, 2],
 [11, 3, 1],
 [11, 4, 1],
 [12, 0, 2],
 [12, 1, 2],
 [12, 2, 2],
 [12, 3, 2],
 [12, 4, 2],
 [13, 0, 1],
 [13, 1, 1],
 [13, 2, 1],
 [13, 3, 1],
 [13, 4, 1],
 [14, 0, 2],
 [14, 1, 2],
 [14, 2, 2],
 [14, 3, 2],
 [14, 4, 2],
 [15, 0, 2],
 [15, 1, 2],
 [15, 2

In [33]:
filename = f"./data/{now}/llm_response_{n_sample}__{now}.txt"

with open(filename, "w") as file:
    for sublist in llm_result_list:
        line = " ".join(str(num) for num in sublist)  # Convert to string, join with spaces
        file.write(line + "\n")  # Write line and add newline
filename

'./data/20240530/llm_response_100__20240530.txt'

#  GLAD

In [34]:
from utils import glad

In [35]:
glad_output = glad(filename)

# Eval

In [ ]:
import json
import pandas as pd
from collections import Counter
from typing import Dict, List
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
def get_majority_vote(label_dict: Dict[str, List[int]]) -> List[int]:
    """
    Finds the majority value for each element across multiple lists within a dictionary.

    Args:
        label_dict: A dictionary where keys are identifiers and values are lists of labels.

    Returns:
        A list of majority values corresponding to each element position.
    """
    list_of_labels = list(label_dict.values())  # Extract values into a list
    majority_values = []

    for elements in zip(*list_of_labels):
        element_counts = Counter(elements)
        most_common_element = element_counts.most_common(1)[0]
        majority_values.append(most_common_element[0])

    return majority_values


def accuracy_with_none_penalty(y_true, y_pred):
    filtered_y_true = []
    filtered_y_pred = []

    for true, pred in zip(y_true, y_pred):
        if pred is not None:  # Only include non-None predictions
            filtered_y_true.append(true)
            filtered_y_pred.append(pred)
        else:
            filtered_y_true.append(true)  # Include true label
            filtered_y_pred.append(-1)   # Replace None with wrong label (e.g., -1)

    return accuracy_score(filtered_y_true, filtered_y_pred)

In [ ]:
with open("./data/raw_llm_response_100__20240529.json", "r") as f:
    llm_response = json.load(f)

df_glad = pd.read_csv("./data/label_glad__20240529.csv")


In [ ]:
llm_response["majority"] = get_majority_vote(llm_response)
llm_response["glad"] = df_glad["label"].values

In [ ]:
for k, v in llm_response.items():
    print(k, accuracy_with_none_penalty(dataset['answer'], v))

In [ ]:

return {"accuracy": accuracy_score(y_true, y_pred),
                "f1_weighted": f1_score(y_true, y_pred, average='weighted'),
                "confusion_matrix": confusion_matrix(y_true, y_pred)
                }